In [1]:
import numpy as np

import json
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
import keras.backend as K
import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with open('data/sentipolc_char_index.json', 'r') as f:
    word_index = json.load(f)

In [3]:
with open('data/chars.tsv', 'w') as f:
    for w,_ in word_index.items():
        f.write('{}\n'.format(w))

In [4]:
data = np.load('data/sentipolc_char_seq.npz')

x_train = data['x_train']
y_train = data['y_train']

x_test = data['x_test']
y_test = data['y_test']

Check the max length of the text

In [5]:
max_len_seq = max([len(x) for x in x_train])
print('max len seq {}'.format(max_len_seq))
max_idx = max(word_index.values())
print('max id {}'.format(max_idx))

max len seq 190
max id 127


In [6]:
x_train_pad = sequence.pad_sequences(x_train, maxlen=max_len_seq)
x_test_pad = sequence.pad_sequences(x_test, maxlen=max_len_seq)

In [7]:
x_train_pad.shape

(4154, 190)

# Define metrics

In [8]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.
     Computes the precision, a metric for multi-label classification of
     how many selected items are relevant.
      """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [9]:
def recall(y_true, y_pred):
    """Recall metric.
 
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [10]:
from sklearn.model_selection import StratifiedShuffleSplit

split_train_val = StratifiedShuffleSplit(1,test_size=0.2, random_state=45)

for train, test in split_train_val.split(x_train_pad, y_train):
    x_train_pad_split, y_train_pad_split = x_train_pad[train], y_train[train]
    x_val_pad_split, y_val_pad_split = x_train_pad[test], y_train[test]

In [11]:

callbacks = [
    ReduceLROnPlateau(patience=2, verbose=1),
    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')
]

In [14]:
def build_model():
    model = Sequential()
    model.add(Embedding(input_dim=max_idx+1, output_dim=100, input_length=max_len_seq))
    model.add(GRU(32,  dropout=0.0, recurrent_dropout=0.0, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy', recall, precision])
    
    return model

In [15]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 190, 100)          12800     
_________________________________________________________________
gru_2 (GRU)                  (None, 32)                12768     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 25,601
Trainable params: 25,601
Non-trainable params: 0
_________________________________________________________________


In [16]:
history = model.fit(x_train_pad_split, y_train_pad_split, 
                    validation_data=(x_val_pad_split, y_val_pad_split),
                    batch_size=32, epochs=10, callbacks=callbacks)

Train on 3323 samples, validate on 831 samples
Epoch 1/10
3323/3323 [==============================] - 17s 5ms/step - loss: 0.6548 - acc: 0.6302 - recall: 0.0640 - precision: 0.3887 - val_loss: 0.6377 - val_acc: 0.6510 - val_recall: 0.1498 - val_precision: 0.6624
Epoch 2/10
3323/3323 [==============================] - 16s 5ms/step - loss: 0.6255 - acc: 0.6681 - recall: 0.2483 - precision: 0.7204 - val_loss: 0.6243 - val_acc: 0.6655 - val_recall: 0.2143 - val_precision: 0.6919
Epoch 3/10
3323/3323 [==============================] - 16s 5ms/step - loss: 0.6150 - acc: 0.6732 - recall: 0.2621 - precision: 0.7156 - val_loss: 0.6189 - val_acc: 0.6691 - val_recall: 0.2837 - val_precision: 0.6708
Epoch 4/10
3323/3323 [==============================] - 16s 5ms/step - loss: 8.9348 - acc: 0.4225 - recall: 0.9465 - precision: 0.4320 - val_loss: 9.7650 - val_acc: 0.3875 - val_recall: 1.0000 - val_precision: 0.3875
Epoch 5/10
3323/3323 [==============================] - 16s 5ms/step - loss: 9.7583 -

## Evaluate on the test set

In [17]:
callbacks = [
    TensorBoard(histogram_freq=0, batch_size=128, embeddings_freq=1, embeddings_metadata=words),
]

NameError: name 'words' is not defined

In [18]:
model = build_model()

history = model.fit(x_train_pad, y_train, batch_size=128, epochs=6, callbacks=callbacks)

Epoch 1/6
4154/4154 [==============================] - 8s 2ms/step - loss: 0.6773 - acc: 0.6134 - recall: 0.1373 - precision: 0.7344
Epoch 2/6


/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:958: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,recall,precision,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,recall,precision,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


4154/4154 [==============================] - 7s 2ms/step - loss: 0.6447 - acc: 0.6449 - recall: 0.1110 - precision: 0.7232
Epoch 3/6
4154/4154 [==============================] - 7s 2ms/step - loss: 0.6264 - acc: 0.6649 - recall: 0.2281 - precision: 0.7170
Epoch 4/6
4154/4154 [==============================] - 7s 2ms/step - loss: 0.6186 - acc: 0.6675 - recall: 0.2329 - precision: 0.7244
Epoch 5/6
4154/4154 [==============================] - 7s 2ms/step - loss: 0.6726 - acc: 0.6726 - recall: 0.2568 - precision: 0.7184
Epoch 6/6
4154/4154 [==============================] - 7s 2ms/step - loss: 0.6132 - acc: 0.6740 - recall: 0.2437 - precision: 0.7480


In [19]:
loss, acc = model.evaluate(x_test_pad, y_test)

1050/1050 [==============================] - 1s 1ms/step


ValueError: too many values to unpack (expected 2)

In [ ]:
print('loss {}, acc {}'.format(loss, acc))

In [ ]:
#model.save('models/sentipolc_char_lstm.hdf5')

## Confusion matrix

In [22]:
from sklearn import metrics

In [20]:
predictions = model.predict_classes(x_train_pad, batch_size=128)

In [23]:
tn, fp, fn, tp = metrics.confusion_matrix(y_train, predictions).ravel()

In [24]:
recall = tp/(tp+fp)
precision = tp/(tp+fn)
f1 = metrics.f1_score(y_train,predictions)

print ('precision: {}'.format(precision), '\nrecall: {}'.format(recall), '\nf1: {}'.format(f1))

precision: 0.25201738050900063 
recall: 0.7422303473491774 
f1: 0.37627432808155703
